In [30]:
# Dependencies
import json
from numpy import random
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from citipy import citipy
from config import openweather_key

In [31]:
lat = np.random.uniform(low=-90.000, high=90.000, size=1500)
lon = np.random.uniform(low=-180.000, high=180.000, size=1500)

In [32]:
coords = {"Lat": lat, "Lng": lon}
df = pd.DataFrame(coords)

In [33]:
new_df = df.sample(n=500)
new_df.head()

,Lat,Lng
1490,4.124799,-94.487270
551,-81.384471,-76.750947
1204,-88.232789,172.640784
733,85.761617,16.120475
418,60.160656,-139.360963


In [34]:
new_df = new_df.reset_index(drop=True)
new_df.head()

,Lat,Lng
0,4.124799,-94.487270
1,-81.384471,-76.750947
2,-88.232789,172.640784
3,85.761617,16.120475
4,60.160656,-139.360963


In [35]:
coordinates = tuple(zip(new_df["Lat"], new_df["Lng"]))
coordinates

((4.1247990413005056, -94.487270447970033),
 (-81.384471155823988, -76.750946601077686),
 (-88.232788766903141, 172.64078415216937),
 (85.761617309634914, 16.120474862548519),
 (60.160656093865299, -139.36096251118613),
 (78.257651580544575, -50.106137188624444),
 (85.431805409567147, 88.790564556502375),
 (-33.760466212132066, -134.78109635514986),
 (-66.262286275339534, -164.38832440967732),
 (-44.830892076364705, 80.587713377520288),
 (80.587629136709637, -83.508427323629718),
 (16.525127761250829, -9.170011258701777),
 (53.688729306573947, 6.8405251882823563),
 (38.264196366215941, 138.60208965691641),
 (73.751330037671522, -39.594434525224045),
 (-29.535267750216541, -67.592609613937697),
 (65.374373633189151, 9.9098825472364922),
 (51.549051226149288, -136.49080602834829),
 (87.394908872512758, 125.666542505422),
 (8.131135875792026, 78.403655450005658),
 (-1.5184465543510299, -139.10278035416519),
 (-87.537100730349508, 156.44132224547758),
 (-37.393688509044843, 70.012891631071

In [23]:
cities = []
city_name = []
for coordinate_pair in coordinates:
    lat, lon = coordinate_pair
    cities.append(citipy.nearest_city(lat, lon))

for city in cities:
    country_code = city.country_code
    name = city.city_name
    city_name.append(name)

In [24]:
new_df["City_Name"] = city_name
new_df.head()

,Lat,Lng,City_Name
0,67.943839,77.669151,tazovskiy
1,35.486866,132.087205,gotsu
2,50.203392,4.044951,maubeuge
3,7.390646,39.456859,asasa
4,59.222474,-99.288561,thompson


In [25]:
# OpenWeather config info
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

In [26]:
# Build query URL
query_url = url + "appid=" + openweather_key + "&units=" + units + "&q=" + "Paris"

In [27]:
response = requests.get(query_url)
response.json()

{'base': 'stations',
 'clouds': {'all': 90},
 'cod': 200,
 'coord': {'lat': 48.85, 'lon': 2.35},
 'dt': 1511031600,
 'id': 2988507,
 'main': {'humidity': 87,
  'pressure': 1023,
  'temp': 45.88,
  'temp_max': 48.2,
  'temp_min': 44.6},
 'name': 'Paris',
 'sys': {'country': 'FR',
  'id': 5617,
  'message': 0.1831,
  'sunrise': 1510988731,
  'sunset': 1511021155,
  'type': 1},
 'visibility': 10000,
 'weather': [{'description': 'overcast clouds',
   'icon': '04n',
   'id': 804,
   'main': 'Clouds'}],
 'wind': {'deg': 230, 'speed': 9.17}}

In [28]:
# Create empty column for desired data
new_df["Temperature"] = ""
new_df["Humidity"] = ""
new_df["Cloudiness"] = ""
new_df["Wind_Speed"] = ""
weather_data = []
units = "imperial"

# Counter
row_count = 0

# Loop through and run Google search to get all banks in 5 mile radius (8000 meters)
for index, row in new_df.iterrows():
    
    # Create endpoint url using Google Places Radar and the lat/lng we identified earlier
    target_url = url + "appid=" + openweather_key + "&units=" + units + "&q=" + str(new_df.loc[index]["City_Name"])

    # This link helps to handily see the JSON generated for each query
    print("Now retrieving city #%s: %s" % (row_count, new_df.loc[index]["City_Name"]))
    row_count += 1 
    print(target_url)
    
    # Run a request to grab the JSON at the target URL
    response = requests.get(target_url).json()
    weather_data.append(response)
    
    # Extract interesting data from responses
    temp_data = [data.get("main").get("temp") for data in weather_data]
    humidity_data = [data.get("main").get("humidity") for data in weather_data]
    cloud_data = [data.get("clouds").get("all") for data in weather_data]
    wind_data = [data.get("wind").get("speed") for data in weather_data]

    
    # Store the bank count into the Data Frame
    new_df.set_value(index, "Temperature", temp_data[-1])
    new_df.set_value(index, "Humidity", humidity_data[-1])
    new_df.set_value(index, "Cloudiness", cloud_data[-1])
    new_df.set_value(index, "Wind_Speed", wind_data[-1])

    # Reset Counts
    temp_data = []
    humidity_data = []
    cloud_data = []
    wind_data = []
    
# Visualize
new_df.head()

Now retrieving city #0: tazovskiy
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=tazovskiy
Now retrieving city #1: gotsu
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=gotsu
Now retrieving city #2: maubeuge
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=maubeuge
Now retrieving city #3: asasa
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=asasa
Now retrieving city #4: thompson
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=thompson
Now retrieving city #5: bundaberg
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=bundaberg
Now retrieving city #6: lerwick
http://api.openweathermap.org/data/2.5/weather?appid=e566eb70a6b9d0b1e7f80b12308dafea&units=imperial&q=ler

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
# Build a scatter plot for each data type
plt.scatter(new_df["Lat"], new_df["Temperature"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude vs. Max Temperature")
plt.ylabel("Temperature")
plt.xlabel("Latitude")
plt.grid(True)


# Show plot
plt.show()

In [ ]:
# Build a scatter plot for each data type
plt.scatter(new_df["Lat"], new_df["Humidity"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity")
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.grid(True)


# Show plot
plt.show()

In [ ]:
# Build a scatter plot for each data type
plt.scatter(new_df["Lat"], new_df["Cloudiness"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)


# Show plot
plt.show()

In [29]:
# Build a scatter plot for each data type
plt.scatter(new_df["Lat"], new_df["Wind_Speed"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed")
plt.ylabel("Wind Speed")
plt.xlabel("Latitude")
plt.grid(True)


# Show plot
plt.show()

ValueError: could not convert string to float: 